In [1]:
import gempyor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
import copy

# import seaborn as sns
import matplotlib._color_data as mcd
import pyarrow.parquet as pq
import click
import subprocess
import dask.dataframe as dd
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from matplotlib.backends.backend_pdf import PdfPages
from gempyor.utils import config, as_list
import os
import multiprocessing as mp
import pandas as pd
import pyarrow.parquet as pq
import xarray as xr
from gempyor import config, model_info, outcomes, seir, inference_parameter, logloss, inference
from gempyor.inference import GempyorInference
import gempyor.postprocess_inference

import os
from multiprocessing import cpu_count

# disable  operations using the MKL linear algebra.
os.environ["OMP_NUM_THREADS"] = "1"
import emcee

from multiprocessing import Pool

## Test A
project_path = "./flepiMoP/examples/simple_usa_statelevel/"  # necessary for now for time serie parameters !!! FIXME
config_filepath = f"simple_usa_statelevel.yml"
seir_modifiers_scenario="Ro_all"
outcome_modifiers_scenario=None

# Test B
#project_path = "RSV_USA/"
#config_filepath = f"config_rsvnet_emcee.yml"
#seir_modifiers_scenario="none"
#outcome_modifiers_scenario="IHRadj"

# Test C
project_path = "COVID19_USA/"
config_filepath= "config_SMH_R18_allBoo_highIE_blk1_emcee.yml"
#config_filepath="config_FCH_R17_allBoo_highIE_phase5_blk1_21Apr2024.yml"


nwalkers = 3
niter = 4
nsamples = 4 # number of likelyhood eval to write to disk...
nthin=5
run_id = config_filepath.split(".")[0]

#### START COPY PAST FROM flepiMoP/flepimop/gempyor_pkg/src/gempyor/calibrate.py
ncpu = cpu_count()
print(f"found {ncpu} CPUs, using these")

gempyor_inference = GempyorInference(
        config_filepath=config_filepath,
        run_id=run_id,
        prefix=None,
        first_sim_index=1,
        stoch_traj_flag=False,
        rng_seed=None,
        nslots=1,
        inference_filename_prefix="",  # usually for {global or chimeric}/{intermediate or final}
        inference_filepath_suffix="",  # usually for the slot_id
        out_run_id=None,  # if out_run_id is different from in_run_id, fill this
        out_prefix=None,  # if out_prefix is different from in_prefix, fill this
        path_prefix=project_path,  # in case the data folder is on another directory
        autowrite_seir=False,
)

#inferpar = inference_parameter.InferenceParameters(global_config=config, modinf=modinf)
p0 = gempyor_inference.inferpar.draw_initial(n_draw=nwalkers)
for i in range(nwalkers):
    assert  gempyor_inference.inferpar.check_in_bound(proposal=p0[i]), "The initial parameter draw is not within the bounds, check the perturbation distributions"

nsubpop = len(gempyor_inference.modinf.subpop_struct.subpop_names)
subpop_names = gempyor_inference.modinf.subpop_struct.subpop_names


print(gempyor_inference.logloss)
print(gempyor_inference.inferpar)

found 12 CPUs, using these
  gempyor >> Running ***DETERMINISTIC*** simulation;
  gempyor >> ModelInfo USA_inference_all; index: 1; run_id: config_SMH_R18_allBoo_highIE_blk1_emcee,
  gempyor >> prefix: USA/config_SMH_R18_allBoo_highIE_blk1_emcee/;
Loaded subpops in loaded relative probablity file: 51 Intersect with seir simulation:  51 kept
ERROR: subpop_names 25000 (idx: pl_idx) has a population from initial condition of 6850563.049999996 while population from geodata is 6850553 (absolute difference should be < 1, here is 10.049999996088445)
 Ignoring the previous population mismatch errors because you added flag 'ignore_population_checks'. This is dangerous
Running Gempyor Inference
LogLoss: 2 statistics and 10710 data points,number of NA for each statistic: 
source        0
incidC    10710
cumC      10710
incidD        0
cumD          0
incidH       51
cumH         51
dtype: int64
InferenceParameters: with 717 parameters: 
    seir_modifiers: 615 parameters
    outcome_modifiers: 10

In [2]:
filename = f"{run_id}_backend.h5"
sampler = emcee.backends.HDFBackend(filename, read_only=True)


In [12]:
n = 1
max_indices = np.argsort(sampler.get_log_prob()[-1, :])[-n:]
max_indices

array([1])

In [2]:
sampler.get_chain().shape

(2, 3, 717)

In [29]:
samples = sampler.get_chain()[-1,max_indices ,:]
a = [(samples[i,:],) for i in range(len(max_indices))]

In [3]:
gempyor_inference.perform_test_run()

test run successful 🎉, with logloss=-45965.6 including -5500.4 for regularization (12.0%) 


(           subpop  incidI_unvaccinated_ALL_age0to17ph4  \
 date                                                     
 2023-03-19  56000                           110.394339   
 2023-03-20  56000                           110.394339   
 2023-03-21  56000                           132.331477   
 2023-03-22  56000                           132.331477   
 2023-03-23  56000                           142.058054   
 ...           ...                                  ...   
 2025-08-27  06000                         16035.451907   
 2025-08-28  06000                         16035.451907   
 2025-08-29  06000                         16634.840639   
 2025-08-30  06000                         16634.840639   
 2025-08-31  06000                         17256.442292   
 
             incidI_vaccinated_ALL_age0to17ph4  \
 date                                            
 2023-03-19                           4.605727   
 2023-03-20                           4.605727   
 2023-03-21                    

In [4]:
filename = f"{run_id}_backend.h5"
backend = emcee.backends.HDFBackend(filename)
backend.reset(nwalkers, gempyor_inference.inferpar.get_dim())
gempyor_inference.set_silent(False)
with Pool(3) as pool:
        sampler = emcee.EnsembleSampler(nwalkers, 
                                        gempyor_inference.inferpar.get_dim(), 
                                        gempyor_inference.get_logloss_as_single_number, 
                                        pool=pool,
                                        backend=backend, moves=[(emcee.moves.StretchMove(live_dangerously=True), 1)])
        state = sampler.run_mcmc(p0, niter, progress=True, skip_initial_state_check=True)

llik is -45729.70247489614
llik is -45842.41667236088
llik is -45976.52186496037


  0%|          | 0/4 [00:00<?, ?it/s]

llik is -45757.469693276245
llik is -45865.942736300945


 25%|██▌       | 1/4 [00:32<01:38, 32.96s/it]

llik is -45972.74954846854
llik is -45591.36506356454
llik is -45901.314939441305


 50%|█████     | 2/4 [01:05<01:05, 32.74s/it]

llik is -46074.59348286781
llik is -45890.32092923346
llik is -45969.61363397053


 75%|███████▌  | 3/4 [01:38<00:33, 33.00s/it]

llik is -45519.50216712484
llik is -45544.68410035816
llik is -45883.33418188836


In [ ]:
# config: config_SMH_R18_allBoo_highIE_blk1_emcee.yml
p_draw = modinf.parameters.parameters_quick_draw(n_days=modinf.n_days, nsubpops=modinf.nsubpops)
npi_seir = seir.build_npi_SEIR(modinf=modinf, load_ID=False, sim_id2load=None, config=config, bypass_DF=gempyor.read_df("a/000000005.FCH_R17_allBoo_highIE_phase5_blk1_21Apr2024.snpi.parquet"))
parameters = modinf.parameters.parameters_reduce(p_draw, npi_seir)
parsed_parameters = modinf.compartments.parse_parameters(parameters, modinf.parameters.pnames, static_sim_arguments["unique_strings"])

toplot = 'r0*gamma*theta3*1*chi_OMICRON*1'
toplot_id = static_sim_arguments["unique_strings"].index(toplot)
# try 3 rolling mean with differnt windowns and plot them on the same fig
fig, ax = plt.subplots(8, 1, figsize=(10, 12))
ax[0].set_title(f"parameter {toplot} from slot 5 of FCH_R17_allBoo_highIE_phase5_blk1_21Apr2024")
for i, window in enumerate([7, 1, 20, 31, 38, 45, 65, 80]):
    ax[i].plot(parsed_parameters[toplot_id, :, 0], color = 'k', lw=.5, ls='-.', label = f"current from config")
    #ax[i].plot(pd.Series(parsed_parameters[toplot_id, :, 0]).rolling(window=window, center=True).mean(), label = f"window {window}", lw=3)
    ax[i].plot(gempyor.utils.rolling_mean_pad(data=parsed_parameters[toplot_id, :, :], window=window)[:,0], label = f"window {window}")
    ax[i].set_title(f"mean difference: {np.mean(np.abs(parsed_parameters[toplot_id, :, 0] - gempyor.utils.rolling_mean_pad(data=parsed_parameters[toplot_id, :, :], window=window)[:,0]))}")
    ax[i].legend()
fig.tight_layout()

AttributeError: 'GempyorInference' object has no attribute 'inferpar'

In [ ]:
filename = f"{run_id}_backend.h5"
backend = emcee.backends.HDFBackend(filename)
backend.reset(3, inferpar.get_dim())
with Pool(3) as pool:
        sampler = emcee.EnsembleSampler(3, 
                                        inferpar.get_dim(), 
                                        a.run, 
                                        pool=pool,
                                        backend=backend, moves=[(emcee.moves.StretchMove(live_dangerously=True), 1)])
        state = sampler.run_mcmc(p0[0:3,:], 5, progress=True, skip_initial_state_check=True)

llik: -46237.56540759982
llik: -46051.281202656246
llik: -45831.8716673561


  0%|          | 0/5 [00:00<?, ?it/s]

llik: -46099.484509051355
llik: -46195.55396420405


 20%|██        | 1/5 [00:33<02:13, 33.33s/it]

llik: -45909.410404363836


 20%|██        | 1/5 [00:34<02:17, 34.33s/it]


KeyboardInterrupt: 

In [ ]:
ds["incidH_18_49"].sum(dim="subpop", skipna=False).plot()

In [ ]:
ds["incidH_18_49"].resample(date="W-SAT").sum().plot.line(x="date");

In [ ]:
ds["incidH_18_49"].resample(date="W-SAT").sum(skipna=False).plot.line(x="date");

In [ ]:
ds["incidH_18_49"].resample(date="W-SAT", skipna=False).sum().plot.line(x="date");

In [ ]:
if False:
    log_loss.plot_gt(subplot=True);
    for reg_func, reg_config in log_loss.statistics['incidCase'].regularizations:
        print(reg_func,"\n", reg_config)

## Create the first gempyor object

In [ ]:
hosp.reset_index().set_index(["date","subpop"])
xr.Dataset.from_dataframe(hosp.reset_index().set_index(["date","subpop"]))

In [ ]:
raise ValueError

## Run EMCEE

In [ ]:
backend = emcee.backends.HDFBackend(filename)
with Pool(ncpu) as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob, 
                                    args=[snpi_df_ref, ndim, statistics, fitted_params, gt, hnpi_df_ref, modinf, p_draw, unique_strings, transition_array, proportion_array, proportion_info, initial_conditions, seeding_data, seeding_amounts,outcomes_parameters, True], 
                                    pool=pool,
                                    backend=backend,  moves=[(emcee.moves.StretchMove(live_dangerously=True), 1)])
    state = sampler.run_mcmc(None, (nsamples//nwalkers)+1, progress=True, skip_initial_state_check=True)

In [ ]:
    print(f"Done, mean acceptance fraction: {np.mean(sampler.acceptance_fraction):.3f}")

In [ ]:
plt.plot(sampler.acceptance_fraction)

## Analysi

In [ ]:
filename = f"{run_id}_backend.h5"
sampler = emcee.backends.HDFBackend(filename, read_only=True)

In [ ]:
sampler.get_chain().shape

In [ ]:
# todo: restart form other shape...

In [ ]:
last_llik = sampler.get_log_prob()[-1,:]
good_slots = last_llik > (last_llik.mean()-1*last_llik.std())
print(f"there are {good_slots.sum()}/{len(good_slots)} good walkers... keeping these")

In [ ]:
fig, axes = plt.subplots(inferpar.get_dim()+1,2, figsize=(15, (inferpar.get_dim()+1)*2))

labels = list(zip(inferpar.pnames, inferpar.subpops))
samples = sampler.get_chain()

import seaborn as sns
def plot_chain(frompt,axes):
    ax = axes[0]

    ax.plot(np.arange(frompt,frompt+sampler.get_log_prob()[frompt:].shape[0]),
                      sampler.get_log_prob()[frompt:,good_slots], "navy", alpha=.2, lw=1, label="good walkers")
    ax.plot(np.arange(frompt,frompt+sampler.get_log_prob()[frompt:].shape[0]),
            sampler.get_log_prob()[frompt:,~good_slots], "tomato", alpha=.4, lw=1, label="bad walkers")
    ax.set_title("llik")
    #ax.legend()
    sns.despine(ax=ax, trim=False)
    ax.set_xlim(frompt, frompt+sampler.get_log_prob()[frompt:].shape[0])

    #ax.set_xlim(0, len(samples))

    for i in range(inferpar.get_dim()):
        ax = axes[i+1]
        ax.plot(np.arange(frompt,frompt+sampler.get_log_prob()[frompt:].shape[0]),
                samples[frompt:,good_slots, i], "navy", alpha=.2, lw=1,)
        ax.plot(np.arange(frompt,frompt+sampler.get_log_prob()[frompt:].shape[0]),
                samples[frompt:, ~good_slots, i], "tomato", alpha=.4, lw=1,)
        #ax.set_xlim(0, len(samples))
        ax.set_title(labels[i])
        #ax.yaxis.set_label_coords(-0.1, 0.5)
        sns.despine(ax=ax, trim=False)
        ax.set_xlim(frompt, frompt+samples[frompt:].shape[0])
        

    axes[-1].set_xlabel("step number");

plot_chain(0,axes[:,0])
plot_chain(3*samples.shape[0]//4,axes[:,1])
fig.tight_layout()

plt.savefig(f"{run_id}_chains.pdf")

In [ ]:
good_samples =  sampler.get_chain()[:,good_slots,:]
good_samples.shape

In [ ]:
step_number = -1
exported_samples = np.empty((nsamples,ndim))
for i in range(nsamples):
    exported_samples[i,:] = good_samples[step_number - nthin*(i//(good_slots.sum())) ,i%(good_slots.sum()),:] # parentesis around i//(good_slots.sum() are very important

In [ ]:
position_arguments = [snpi_df_ref, ndim, statistics, fitted_params, gt, hnpi_df_ref, modinf, p_draw, unique_strings, transition_array, proportion_array, proportion_info, initial_conditions, seeding_data, seeding_amounts,outcomes_parameters, True]

with Pool(ncpu) as pool:
    results = pool.starmap(log_prob, [(sample, *position_arguments) for sample in exported_samples])

In [ ]:
np.mean(sampler.get_log_prob()[-1,:])

In [ ]:
#import concurrent.futures
#
def evaluate(proposals, k):
    proposal = proposals[k]
    snpi_df_mod, hnpi_df_mod = input_proposal(proposal, snpi_df_ref, hnpi_df_ref, fitted_params, ndim)

    outcomes_df = run_simulation(snpi_df_mod, 
                                hnpi_df_mod,
                                modinf, p_draw, unique_strings, transition_array, proportion_array, proportion_info, initial_conditions, seeding_data, seeding_amounts,outcomes_parameters)
    return outcomes_df
#
#with concurrent.futures.ThreadPoolExecutor(max_workers=nwalkers) as executor:
#    # Map the function to the list of k values in parallel
#    results = list(executor.map(lambda k_val: evaluate(samples[-1,:,:], k_val), np.arange(nwalkers)))
#

results = []
for k in range(nwalkers):
    results.append(evaluate(samples[-1,:,:], k))

In [ ]:
results

In [ ]:
results = []
for fn in gempyor.utils.list_filenames(folder="model_output/", filters=[run_id,"hosp.parquet"]):
    if "000000000" not in fn:
        df = gempyor.read_df(fn)
        #raise ValueError
        df["date"] = df["time"]
        df = df.set_index("date")
        results.append(df)

In [ ]:
results[0].index.max()

In [ ]:
fig, axes = plt.subplots(len(statistics),len(subpop_names), figsize=(5*len(subpop_names), 3*len(statistics)), sharex=True, dpi=300)
for j, subpop in enumerate(modinf.subpop_struct.subpop_names):
        gt_s = gt[gt["subpop"]==subpop].sort_index()
        first_date = max(gt_s.index.min(),results[0].index.min())
        last_date = min(gt_s.index.max(), results[0].index.max())
        gt_s = gt_s.loc[first_date:last_date].drop(["subpop"],axis=1).resample("W-SAT").sum()
        
        for i, (key, value) in enumerate(statistics.items()):
                ax = axes[i,j]
                ax.plot(gt_s[value], color='k', marker='.', lw=1)
                for model_df in results:
                        model_df_s = model_df[model_df["subpop"]==subpop].drop(["subpop","time"],axis=1).loc[first_date:last_date].resample("W-SAT").sum() # todo sub subpop here
                        ax.plot(model_df_s[key],  lw=.9, alpha=.5)
                #if True:
                #        init_df_s = outcomes_df_ref[model_df["subpop"]==subpop].drop(["subpop","time"],axis=1).loc[min(gt_s.index):max(gt_s.index)].resample("W-SAT").sum() # todo sub subpop here
                ax.set_title(f"{value}, {subpop}")
fig.tight_layout()
plt.savefig(f"{run_id}_results.pdf")

In [ ]:
gt_s.isna().sum()

In [ ]:
gt_s.plot(subplots=True)

In [ ]:
gt_s["incidH_65_100"]

In [ ]:
gt_s

In [ ]:
emcee.__version__

In [ ]:
import corner

In [ ]:
#tau = sampler.get_autocorr_time()
#print(tau)

flat_samples = sampler.get_chain(discard=30, thin=2,flat=True) #discard=100, thin=15, 
print(flat_samples.shape)

import corner

fig = corner.corner(
    flat_samples, labels=labels, #truths=[m_true, b_true, np.log(f_true)]
);